# Лабораторна робота № 4.

Тема:

Дослідження особливостей реалізації існуючих програмних систем,
які використовують криптографічні механізми захисту інформації.

Мета роботи:

 Отримання практичних навичок побудови гібридних
криптосистем.

Для другого типу лабораторних робіт – розробити реалізацію
асиметричної криптосистеми у відповідності до стандартних вимог Crypto API
або стандартів PKCS та дослідити стійкість стандартних криптопровайдерів до
атак, що використовують недосконалість механізмів захисту операційної
системи.

Варіанти завдань другого типу.

Підгрупа 2А. Бібліотека OpenSSL під Windows платформу. Кр/с Ель Гамаля.
[1] с. 535.

Підгрупа 2B. Бібліотека PyCrypto під Linux платформу. Стандарт ECDSA.

Підгрупа 2C. Бібліотека PyCrypto під Crypto++ під Android/MacOs/Ios
платформу. Реалізація несуперечного цифрового підпису.

Оформлення результатів: контрольний приклад роботи з асиметричною
криптосистемою. Приклад атаки або демонстрація їх неможливості.

In [1]:
!!pip install pycryptodome

['Collecting pycryptodome',
 '  Downloading pycryptodome-3.21.0-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)',
 'Downloading pycryptodome-3.21.0-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.3 MB)',
 '\x1b[?25l   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/2.3 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K   \x1b━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.2/2.3 MB\x1b \x1b8.3 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.9/2.3 MB\x1b \x1b13.1 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━\x1b \x1b2.2/2.3 MB\x1b \x1b19.4 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b2.3/2.3 MB\x1b \x1b18.2 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b2.3/2.3 MB\x1b \x1b13.5 MB/s\x1b eta \x1b0:

In [2]:
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS
from Crypto.Hash import SHA256
import os

def generate_key_pair():
    """Генерує пару ключів ECC"""
    key = ECC.generate(curve='P-256')
    private_key = key.export_key(format='PEM')
    public_key = key.public_key().export_key(format='PEM')
    return private_key, public_key

def save_keys_to_files(private_key, public_key):
    """Зберігає ключі у файли"""
    with open("private_key.pem", "wb") as priv_file:
        priv_file.write(private_key.encode())
    with open("public_key.pem", "wb") as pub_file:
        pub_file.write(public_key.encode())

def sign_message(private_key_path, message):
    """Підписує повідомлення за допомогою приватного ключа"""
    with open(private_key_path, "rt") as key_file:
        private_key = ECC.import_key(key_file.read())

    h = SHA256.new(message.encode())
    signer = DSS.new(private_key, 'fips-186-3')
    signature = signer.sign(h)
    return signature

def verify_signature(public_key_path, message, signature):
    """Перевіряє підпис повідомлення за допомогою публічного ключа"""
    with open(public_key_path, "rt") as key_file:
        public_key = ECC.import_key(key_file.read())

    h = SHA256.new(message.encode())
    verifier = DSS.new(public_key, 'fips-186-3')
    try:
        verifier.verify(h, signature)
        return True
    except ValueError:
        return False

In [3]:
print("Генерація ключів...")
private_key, public_key = generate_key_pair()
save_keys_to_files(private_key, public_key)
print("Ключі збережено у файли private_key.pem та public_key.pem.")

message = "Це тестове повідомлення для підпису."
print("Повідомлення для підпису:", message)

signature = sign_message("private_key.pem", message)
print("Підпис згенеровано.")

is_valid = verify_signature("public_key.pem", message, signature)
print("Перевірка підпису:", "Підпис вірний" if is_valid else "Підпис недійсний")

Генерація ключів...
Ключі збережено у файли private_key.pem та public_key.pem.
Повідомлення для підпису: Це тестове повідомлення для підпису.
Підпис згенеровано.
Перевірка підпису: Підпис вірний


In [4]:
def attack_example(public_key_path, message, signature):
    """Демонструє атаку шляхом модифікації підпису або повідомлення."""
    print("Оригінальне повідомлення:", message)
    print("Перевірка підпису на оригінальному повідомленні...")
    is_valid_original = verify_signature(public_key_path, message, signature)
    print("Результат:", "Підпис вірний" if is_valid_original else "Підпис недійсний")

    # Спроба модифікації повідомлення
    tampered_message = message + " (модифіковане)"
    print("\nМодифіковане повідомлення:", tampered_message)
    is_valid_tampered_message = verify_signature(public_key_path, tampered_message, signature)
    print("Перевірка підпису з модифікованим повідомленням:")
    print("Результат:", "Підпис вірний" if is_valid_tampered_message else "Підпис недійсний")

    # Спроба модифікації підпису
    tampered_signature = bytearray(signature)
    tampered_signature[0] ^= 0x01  # Змінюємо перший байт підпису
    print("\nПеревірка підпису з модифікованим підписом...")
    is_valid_tampered_signature = verify_signature(public_key_path, message, bytes(tampered_signature))
    print("Результат:", "Підпис вірний" if is_valid_tampered_signature else "Підпис недійсний")

In [5]:
attack_example("/content/public_key.pem", message, signature)

Оригінальне повідомлення: Це тестове повідомлення для підпису.
Перевірка підпису на оригінальному повідомленні...
Результат: Підпис вірний

Модифіковане повідомлення: Це тестове повідомлення для підпису. (модифіковане)
Перевірка підпису з модифікованим повідомленням:
Результат: Підпис недійсний

Перевірка підпису з модифікованим підписом...
Результат: Підпис недійсний


In [6]:
def replay_attack_example(public_key_path, original_message, signature):
    """Демонструє атаку повторного використання підпису."""
    print("\n=== Атака повторного використання підпису ===")
    new_message = "Це інше повідомлення, але використовується старий підпис."
    print("Нове повідомлення:", new_message)
    is_valid_replay = verify_signature(public_key_path, new_message, signature)
    print("Результат:", "Підпис вірний" if is_valid_replay else "Підпис недійсний")

In [7]:
replay_attack_example("/content/public_key.pem", message, signature)


=== Атака повторного використання підпису ===
Нове повідомлення: Це інше повідомлення, але використовується старий підпис.
Результат: Підпис недійсний


**Принцип роботи ECC**

Еліптичні криві засновані на задачі дискретного логарифма (ECDLP):
- Відкрита точка Q обчислюється як Q = d * G, де G - генератор кривої, d - приватний ключ.
- Для зловмисника обчислення d із Q без надзвичайно потужних ресурсів є практично неможливим.
- Ця складність базується на перевіреній криптографічній теорії.

Для ключа довжиною 256 біт, необхідні ресурси:
- $2^{128}$ операцій для атаки грубою силою.
